# IMPORTS


In [1]:
from pathlib import Path
from IPython.display import Image
import sys
from functools import cache
from utils.bayesian_optimisation import (
    HDD_PATH,
    FIDELITY_RANGE,
    get_training_data,
    get_candidate_solutions,
    preprocess_features,
    regression_pipeline,
    get_mean_and_std_from_model,
    expected_improvement,
    upper_confidence_bound,
    get_next_scenario_seed_from_aq,
    get_random_scenario_seed,
)

import pandas as pd
from typing import Tuple

import numpy as np

from metadrive.engine.logger import get_logger

logger = get_logger()

# MAIN


In [2]:
train_df = get_training_data()
candidates = get_candidate_solutions()

[INFO] Loading benchmarking data


In [3]:
X_train = preprocess_features(train_df)

In [4]:
y_train = train_df["eval.driving_score"]

In [5]:
target_fidelity = max(FIDELITY_RANGE)
current_best = y_train.xs(target_fidelity).min()
logger.info(f"Current best score is: {current_best:.3f}")

[INFO] Current best score is: 0.063


In [6]:
pipe = regression_pipeline(X_train)
pipe.set_params(regressor__n_jobs=16)
model = pipe.fit(X_train, y_train)
logger.info(f"Model trained")

[INFO] Model trained


In [7]:
# find best candidate in high fidelity
candidates = candidates[~candidates.index.isin(train_df.index.get_level_values("def.seed"))]
logger.info(f"Considering next scenario from {len(candidates)} candidates.")

[INFO] Considering next scenario from 100000 candidates.


In [8]:
hf_test = preprocess_features(candidates)
# test candidates must be in highfidelity beacouse we want to predict hf score
hf_test["fid.ads_fps"] = target_fidelity
hf_test = hf_test[X_train.columns]

In [9]:
mean, std = get_mean_and_std_from_model(model, hf_test)
logger.info(f"Best from model: {mean.min():.3f}")

[INFO] Best from model: 0.294


In [10]:
aq_type = "ei"
match aq_type:
    case "ei":
        aq = expected_improvement(mean, std, current_best)
    case "ucb":
        aq = upper_confidence_bound(mean, std)
    case _:
        raise ValueError("Invalid acquisition function")

[INFO] Maximum EI: -0.231
[INFO] Maximum positive EI: 0.000
[INFO] Maximum EI with uncertainty: 0.392


In [11]:
next_seed = get_next_scenario_seed_from_aq(aq, candidates)
logger.info(f"Next seed to evaluate: {next_seed}")

[INFO] Next seed to evaluate: 1049612


In [12]:
# Project next candidate to all the fidelities
next_cadidate = candidates.loc[[next_seed]]
next_cadidate

,fid.ads_fps,fid.world_fps,def.spawn_lane_index,def.distance,def.max_steps,time.init_time,time.agent_time,time.scenario_time,time.closing_time,def.map_seq.0.id,...,def.vehicles_data.vehicle_36_position_x,def.vehicles_data.vehicle_36_position_y,def.vehicles_data.vehicle_36_position_z,def.vehicles_data.vehicle_36_type,def.vehicles_data.vehicle_36_heading_theta,def.vehicles_data.vehicle_36_length,def.vehicles_data.vehicle_36_width,def.vehicles_data.vehicle_36_height,def.vehicles_data.vehicle_36_spawn_road,def.vehicles_data.vehicle_36_destination
def.seed,,,,,,,,,,,,,,,,,,,,,
1049612,60,60,0,544.867698,16346,0.485539,0,1.520000e-07,2.51674,I,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None


In [13]:
mf_candidates = pd.concat([next_cadidate] * len(FIDELITY_RANGE))
mf_candidates["fid.ads_fps"] = FIDELITY_RANGE

In [14]:
mf_test = mf_candidates.reset_index()[X_train.columns]
mf_test

,fid.ads_fps,def.spawn_lane_index,def.distance,def.max_steps,def.map_seq.1.radius,def.map_seq.1.angle,def.map_seq.1.length,def.map_seq.1.dir,def.map_seq.1.id,def.map_seq.1.decrease_increase,...,def.vehicles_data.vehicle_36_position_x,def.vehicles_data.vehicle_36_position_y,def.vehicles_data.vehicle_36_position_z,def.vehicles_data.vehicle_36_type,def.vehicles_data.vehicle_36_heading_theta,def.vehicles_data.vehicle_36_length,def.vehicles_data.vehicle_36_width,def.vehicles_data.vehicle_36_height,def.vehicles_data.vehicle_36_spawn_road,def.vehicles_data.vehicle_36_destination
0,10,0,544.867698,16346,10.0,NaN,NaN,NaN,X,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None
1,20,0,544.867698,16346,10.0,NaN,NaN,NaN,X,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None
2,30,0,544.867698,16346,10.0,NaN,NaN,NaN,X,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None
3,60,0,544.867698,16346,10.0,NaN,NaN,NaN,X,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None


In [15]:
predicted_dscore, _ = get_mean_and_std_from_model(model, mf_test)

In [16]:
predictions = dict(zip(FIDELITY_RANGE, predicted_dscore))
predictions

{10: np.float64(0.6191389426791322),
 20: np.float64(0.6249150704586642),
 30: np.float64(0.612561409166463),
 60: np.float64(0.6285678102706558)}

In [17]:
hf_prediction = predictions[target_fidelity]
hf_prediction

np.float64(0.6285678102706558)

In [18]:
# maximum absolute error
epsilon = 0.01
logger.info(str(predictions))
# go into reverse order to pick the lowest fidelity, that has acceptable error
for fid, dscore in predictions.items():
    error = abs(dscore - hf_prediction)

    logger.info(
        f"Considering {fid} FPS with predicted dscore {dscore:.3f}, error: {error:.3f}"
    )

    if error < epsilon:
        logger.info(
            f"Picking fidelity {fid} which has predicted dscore error of {error:.3f}"
        )
        break

[INFO] {10: np.float64(0.6191389426791322), 20: np.float64(0.6249150704586642), 30: np.float64(0.612561409166463), 60: np.float64(0.6285678102706558)}
[INFO] Considering 10 FPS with predicted dscore 0.619, error: 0.009
[INFO] Picking fidelity 10 which has predicted dscore error of 0.009


# BAYESIAN OPTIMISATION ITERATION


In [19]:
def pick_next_fidelity(
    next_cadidate: pd.DataFrame, scenario_features, trained_model, epsilon=0.01
) -> int:
    """
    Given chosed scenario decide which fidelity is safe to run.
    Returns fidelity.
    """
    logger.info(f"Picking next fidelity!")
    mf_candidates = pd.concat([next_cadidate] * len(FIDELITY_RANGE))
    mf_candidates["fid.ads_fps"] = FIDELITY_RANGE

    mf_X_test = mf_candidates.reset_index()[scenario_features]

    # predict dscore for each fidelity
    predicted_dscore, _ = get_mean_and_std_from_model(trained_model, mf_X_test)

    predictions = dict(zip(FIDELITY_RANGE, predicted_dscore))

    hf_prediction = predictions[max(FIDELITY_RANGE)]
    logger.info(f"Predicted dscore for high fidelity: {hf_prediction:.3f}")
    logger.info(str(predictions))

    # go into increasing fidelity order
    for fid, dscore in predictions.items():
        # maximum absolute error
        error = abs(dscore - hf_prediction)
        logger.info(f"Considering {fid} FPS with predicted {dscore = :.3f}, {error = :.3f}")

        if error < epsilon:
            logger.info(f"Picking fidelity {fid} with dscore error of {error:.3f}")
            return fid

    raise ValueError("No fidelity with acceptable error found")


def bayes_opt_iteration(train_df, aq_type="ei", fidelity="multifidelity") -> Tuple[int, int]:
    """
    Performs a single iteration of Bayesian Otpimisation
    Returns next scenario seed, and next fidelity to run.

    """

    logger.info(f"Entering Bayesian Opt Iteration with parameters:")
    logger.info(f"N training samples {len(train_df)}, {aq_type = }, {fidelity = }")
    target_fidelity = fidelity
    if fidelity == "multifidelity":
        target_fidelity = max(FIDELITY_RANGE)

    # PREPARE TRAINING DATA
    X_train = preprocess_features(train_df)
    y_train = train_df["eval.driving_score"]

    if target_fidelity not in train_df.index.get_level_values("fid.ads_fps"):
        logger.warning(f"Target fidelity is not present in training set.")
        logger.warning(f"Will run target fidelity now!")
        return get_random_scenario_seed(get_candidate_solutions()), target_fidelity

    current_best = y_train.xs(target_fidelity).min()
    logger.info(f"Current best score is: {current_best:.3f}")

    # TRAIN THE MODEL
    pipe = regression_pipeline(X_train)
    logger.info(f"Training using {len(X_train.columns)} features")
    pipe.set_params(regressor__n_jobs=16)
    model = pipe.fit(X_train, y_train)
    logger.debug(f"Model trained")

    # PREPARE TEST DATA
    candidate_scenarios = get_candidate_solutions()
    # Exclude scenarios that have been evaluated (in any fidelity)
    candidate_scenarios = candidate_scenarios[
        ~candidate_scenarios.index.isin(train_df.index.get_level_values("def.seed"))
    ]
    logger.debug(f"Considering next scenario from {len(candidate_scenarios)} candidates.")

    X_test = preprocess_features(candidate_scenarios)
    # test candidates must be casted to target fidelity
    X_test["fid.ads_fps"] = target_fidelity
    X_test = X_test[X_train.columns]

    # PREDICT DSCORE FOR HIGHFIDELITY
    dscore_predictions, std = get_mean_and_std_from_model(model, X_test)
    logger.info(f"Best from model: {dscore_predictions.min():.3f}")

    match aq_type:
        case "ei":
            aq = expected_improvement(dscore_predictions, std, current_best)
        case "ucb":
            aq = upper_confidence_bound(dscore_predictions, std)
        case _:
            raise ValueError("Invalid acquisition function")

    next_seed = int(get_next_scenario_seed_from_aq(aq, candidate_scenarios))
    logger.info(f"Next seed to evaluate: {next_seed}")

    if fidelity != "multifidelity":
        return next_seed, target_fidelity

    logger.debug(f"Multifidelity enabled")

    next_cadidate = candidate_scenarios.loc[[next_seed]]
    next_fidelity = pick_next_fidelity(next_cadidate, X_train.columns, model)
    assert next_fidelity in FIDELITY_RANGE
    return next_seed, next_fidelity


bayes_opt_iteration(get_training_data(), aq_type="ucb", fidelity="multifidelity")

[INFO] Loading benchmarking data
[INFO] Entering Bayesian Opt Iteration with parameters:
[INFO] N training samples 4000, aq_type = 'ucb', fidelity = 'multifidelity'
[INFO] Current best score is: 0.063
[INFO] Training using 461 features
[INFO] Best from model: 0.294
[INFO] Maximum fitness: 0.706
[INFO] Maximum UCB: 0.968
[INFO] Next seed to evaluate: 1007972
[INFO] Picking next fidelity!
[INFO] Predicted dscore for high fidelity: 0.307
[INFO] {10: np.float64(0.25223350850260146), 20: np.float64(0.25978930686164897), 30: np.float64(0.26706157243788403), 60: np.float64(0.30748069489502794)}
[INFO] Considering 10 FPS with predicted dscore = 0.252, error = 0.055
[INFO] Considering 20 FPS with predicted dscore = 0.260, error = 0.048
[INFO] Considering 30 FPS with predicted dscore = 0.267, error = 0.040
[INFO] Considering 60 FPS with predicted dscore = 0.307, error = 0.000
[INFO] Picking fidelity 60 with dscore error of 0.000


(1007972, 60)

## Test reading search data


In [20]:
rep_path = Path("/media/olek/2TB_HDD/metadrive-data/searches/bayesopt_ucb/10/0")
train_df = get_training_data(rep_path=rep_path)
target_fidelity = 60

bayes_opt_iteration(train_df, aq_type="ucb", fidelity="multifidelity")

[INFO] Loading search data from /media/olek/2TB_HDD/metadrive-data/searches/bayesopt_ucb/10/0


  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Entering Bayesian Opt Iteration with parameters:
[INFO] N training samples 4, aq_type = 'ucb', fidelity = 'multifidelity'
[WARNING] Target fidelity is not present in training set. (352163620.py:54)
[WARNING] Will run target fidelity now! (352163620.py:55)


(np.int64(1003582), 60)

## Random fidelity simualtion


In [21]:
import random

BUDGET = 600
RANDOM_BUDGET = 0.10 * BUDGET
logger.info(f"Random budget: {RANDOM_BUDGET}")

while RANDOM_BUDGET > 0:
    fid = random.choice(FIDELITY_RANGE)
    logger.info(f"Chosen fidelity {fid}FPS")
    RANDOM_BUDGET -= fid // 10

[INFO] Random budget: 60.0
[INFO] Chosen fidelity 30FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 60FPS
[INFO] Chosen fidelity 20FPS
[INFO] Chosen fidelity 20FPS
[INFO] Chosen fidelity 20FPS
[INFO] Chosen fidelity 30FPS
[INFO] Chosen fidelity 30FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 60FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 60FPS
[INFO] Chosen fidelity 30FPS
[INFO] Chosen fidelity 30FPS
[INFO] Chosen fidelity 60FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 60FPS
[INFO] Chosen fidelity 10FPS
[INFO] Chosen fidelity 60FPS
